In [11]:
#pre-amble

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset



# pre processing to remove unnecessary 'filler' words
stop_words = set(stopwords.words('english'))

print(len(stop_words))
stop_words_list = list(stop_words)
sliced_stop_words = set(stop_words_list[:30])  # slice the first 10 words

print(sliced_stop_words)


2023-02-14 21:27:36.410997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


179
{"hadn't", "don't", 'shouldn', "wouldn't", 'hers', 'only', "you've", 'doing', "doesn't", 'nor', 'll', 'my', 'too', 'if', "you'll", 'same', 'ours', "shan't", 'herself', 'won', "mightn't", "didn't", 'again', 'ourselves', 'his', 'through', "won't", 'for', 'so', 'himself'}


In [12]:
# Load the data
data = pd.read_csv('IMDB Dataset.csv')  # function to load the data

print(np.shape(data))
print(data.columns)

print(data.head())

# split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# print the number of rows in each set
print("Number of rows in the training set: ", len(train_data))
print("Number of rows in the validation set: ", len(val_data))

(50000, 2)
Index(['review', 'sentiment'], dtype='object')
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Number of rows in the training set:  40000
Number of rows in the validation set:  10000


In [13]:
def preprocess(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

In [14]:
# Preprocess the texts
for i in range(len(data)):
    data.loc[i, 'review'] = preprocess(data.loc[i, 'review'])
    data.loc[i, 'sentiment'] = 1 if data.loc[i, 'sentiment'] == 'positive' else 0

In [15]:
print(data.head(10))

                                              review sentiment
0  one reviewers mentioned watching oz episode ho...         1
1  a wonderful little production br br the filmin...         1
2  i thought wonderful way spend time hot summer ...         1
3  basically family little boy jake thinks zombie...         0
4  petter mattei love time money visually stunnin...         1
5  probably favorite movie story selflessness sac...         1
6  i sure would like see resurrection dated seahu...         1
7  this show amazing fresh innovative idea first ...         0
8  encouraged positive comments film i looking fo...         0
9  if like original gut wrenching laughter like m...         1


In [16]:
#first loading train and test data into batches
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(val_data, batch_size=32, shuffle=True)


In [17]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [27]:
for batch in train_dataloader:
    print(batch.keys())
    break

KeyError: 21314

In [26]:
# Train the model
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(len(train_dataloader))

model.to(device)
for epoch in range(10):
    for batch in train_dataloader:
        with torch.enable_grad():
            input_ids = batch['reviews'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['sentiment'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        


1250


KeyError: 28766